In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import NuSVC
from sklearn.metrics import classification_report, accuracy_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [6]:
# Specify the categories you want to load
categories = ['sci.space','rec.autos','sci.electronics','rec.motorcycles']
# Load the subset of the dataset
newsgroups_data = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=42)
# Split the data into train and test sets
train_data, test_data, train_target, test_target = train_test_split(newsgroups_data.data, newsgroups_data.target, test_size=0.2, random_state=42)
# Print the target names
print(newsgroups_data.target_names)
# Print the first few lines of the first document in the training set
print(train_data[0])
# Print the target label of the first document in the training set
print("Target Label:", newsgroups_data.target_names[train_target[0]])

['rec.autos', 'rec.motorcycles', 'sci.electronics', 'sci.space']
From: whit@carson.u.washington.edu (John Whitmore)
Subject: Re: 8x oversampling CD player
Organization: University of Washington, Seattle
Lines: 36
NNTP-Posting-Host: carson.u.washington.edu
Keywords: oversampling, CD, digitized audio

In article <mcmahanC5v942.MKJ@netcom.com> mcmahan@netcom.com (Dave Mc Mahan) writes:
> In a previous article, kolstad@cae.wisc.edu (Joel Kolstad) writes:
>>In article <hcbC5un9L.DD0@netcom.com> hcb@netcom.com (H. C. Bowman) writes:
>>>
>>>I just bought a new portable CD player for the office, and I notice that
>>>it proudly proclaims "8 TIMES OVERSAMPLING" on the box. 

>>The data is only ever read once (barring mistracks and such, of course),
>>and eventually gets turned into 44.1 KHz, 16 bit, two channel data.
>>Oversampling takes two discrete data points, and interpolates n-1 points
>>between them for n times oversampling.  When I asked, people said that the
>>interpolation was not simpl

In [7]:
#category occurrences within test set
category_counts = np.bincount(train_target)
# Get the categories with non-zero occurrences
categories = np.nonzero(category_counts)[0]
counts = 0

# Printoccurrences
for category in categories:
    category_name = newsgroups_data.target_names[category]
    count = category_counts[category]
    counts += count
    print(f"Category {category_name}: {count} occurrences")
    
print(counts)


Category rec.autos: 780 occurrences
Category rec.motorcycles: 800 occurrences
Category sci.electronics: 789 occurrences
Category sci.space: 796 occurrences
3165


In [8]:
print(len(train_target))

3165


In [9]:
print(train_data[3017])
print("Target label:", newsgroups_data.target_names[train_target[3017]])


From: ak296@yfn.ysu.edu (John R. Daker)
Subject: Re: First Bike?? and Wheelies
Organization: St. Elizabeth Hospital, Youngstown, OH
Lines: 24
Reply-To: ak296@yfn.ysu.edu (John R. Daker)
NNTP-Posting-Host: yfn.ysu.edu


In a previous article, jbc9+@andrew.cmu.edu (James Leo Belliveau) says:

> Anyone, 
>
>    I am a serious motorcycle enthusiast without a motorcycle, and to
>put it bluntly, it sucks.  I really would like some advice on what would
>be a good starter bike for me.  I do know one thing however, I need to
>make my first bike a good one, because buying a second any time soon is
>out of the question.  I am specifically interested in racing bikes, (CBR
>600 F2, GSX-R 750).  I know that this may sound kind of crazy
>considering that I've never had a bike before, but I am responsible, a
>fast learner, and in love.  Please give me any advice that you think
>would help me in my search, including places to look or even specific
>bikes that you want to sell me.
>
>    Thanks  :-)

Th

In [10]:


def preprocess_data(data):
    # Remove email headers and footers
    data = re.sub(r'From:.*\n', '', data)
    data = re.sub(r'Subject:.*\n', '', data)
    data = re.sub(r'Organization:.*\n', '', data)
    data = re.sub(r'Lines:.*\n', '', data)
    data = re.sub(r'Keywords:.*\n', '', data)
    data = re.sub(r'NNTP-Posting-Host:.*\n', '', data)
    data = re.sub(r'Article-I.D.:.*\n', '', data)
    data = re.sub(r'X-Newsreader:.*\n', '', data)
    # Remove email addresses
    data = re.sub(r'\S+@\S+', '', data)
    # Remove special characters and punctuation
    data = re.sub(r'[^a-zA-Z0-9\s]', '', data)
    data = data.lower()
    # Remove extra whitespace
    """
    data = re.sub(r'\s+', ' ', data)
    data = data.strip()
    """
    # Remove digits
    data = re.sub(r'\d', '', data)
    
    return data

In [11]:
# Preprocess train data
preprocessed_train_data = [preprocess_data(d) for d in train_data]
# Preprocess test data
preprocessed_test_data = [preprocess_data(d) for d in test_data]
# preprocessed train data
print(preprocessed_train_data)
#preprocessed test data
print(preprocessed_test_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
# preprocessed train data
for i, data in enumerate(preprocessed_train_data):
    print(f"Preprocessed Train Example {i+1}:")
    print(data)
    print()

# preprocessed test data
for i, data in enumerate(preprocessed_test_data):
    print(f"Preprocessed Test Example {i+1}:")
    print(data)
    print()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_text = [word for word in tokens if word.casefold() not in stop_words]
    return ' '.join(filtered_text)

preprocessed_train_data = [remove_stopwords(d) for d in preprocessed_train_data]
preprocessed_test_data = [remove_stopwords(d) for d in preprocessed_test_data]
# Print preprocessed train data
for i, data in enumerate(preprocessed_train_data):
    print(f"Preprocessed Train Example {i+1}:")
    print(data)
    print()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nizar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nizar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Preprocessed Train Example 1:
article dave mc mahan writes previous article joel kolstad writes article h c bowman writes bought new portable cd player office notice proudly proclaims times oversampling box data ever read barring mistracks course eventually gets turned khz bit two channel data oversampling takes two discrete data points interpolates n points n times oversampling asked people said interpolation simply linear interpolation significantly complicated quite correct understanding case care filtering method uses fir finite impulse response filter id guess cd makers use kind method anybody know real answer typical old xoversampling units thirteentap fir implemented dedicated hardware addition circuit kind speed slow digital standards adder much less expensive analog components comparable precision gleaned information hardware manual old cd player design one would find similar info particular cd digital filter interesting component point view idea digital filter kind interpolat

In [14]:
nltk.download('punkt')
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

preprocessed_train_data = [tokenize(d) for d in preprocessed_train_data]
preprocessed_test_data = [tokenize(d) for d in preprocessed_test_data]
# Print preprocessed train data
for i, data in enumerate(preprocessed_train_data):
    print(f"Preprocessed Train Example {i+1}:")
    print(data)
    print()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nizar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:


# Convert the preprocessed train data to a list of strings
train_text = [' '.join(doc) for doc in preprocessed_train_data]
# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()
# Fit the vectorizer on the preprocessed train data
vectorizer.fit(train_text)
# Transform the preprocessed train data into TF-IDF features
train_features = vectorizer.transform(train_text)
# Convert the preprocessed test data to a list of strings
test_text = [' '.join(doc) for doc in preprocessed_test_data]
# Transform the preprocessed test data into TF-IDF features
test_features = vectorizer.transform(test_text)
# Print the shape of the TF-IDF features
print("Shape of TF-IDF features:", train_features.shape)
# Print the feature names (vocabulary)
print("Feature names (vocabulary):")
print(vectorizer.get_feature_names())

Shape of TF-IDF features: (3165, 30905)
Feature names (vocabulary):


AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [16]:
print(len(preprocessed_train_data))

3165


In [ ]:

# Join the tokenized words into strings
preprocessed_train_text = [' '.join(doc) for doc in preprocessed_train_data]
preprocessed_test_text = [' '.join(doc) for doc in preprocessed_test_data]
# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()
# Fit/transform the training data
X_train = vectorizer.fit_transform(preprocessed_train_text)
# Transform the test text 
X_test = vectorizer.transform(preprocessed_test_text)
# instance creation of NuSVC classifier
svm_classifier = NuSVC(kernel='rbf')
# Training SVM classifier
svm_classifier.fit(X_train, train_target)
# labels prediction
y_pred = svm_classifier.predict(X_test)
# perfiormace evaluation
print("Classification Report:")
print(classification_report(test_target, y_pred))
print("Accuracy:", accuracy_score(test_target, y_pred))